# Análise das Emendas PIX com Clusters

Este notebook carrega `data/dados_com_clusters.csv` e realiza as etapas descritas na metodologia do projeto:
1. *Exploração inicial*
2. *Modelagem OLS com efeitos fixos de partido*
3. *Modelagem multinível step-up*


In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm


In [41]:
df = pd.read_csv("../data/dados_com_clusters.csv").dropna()
df = df[df["porcentagem_votos_validos_2024"] < 1]
df = df[df["emendas_pix_per_capita_partido_prefeito_eleito"] > 1000]
df.head()

,municipio,sigla_municipio,prefeito_eleito_2024,sigla_partido_prefeito_eleito,porcentagem_votos_validos_2024,emendas_pix_per_capita_partido_prefeito_eleito,idhm_2010,alfabetizacao_2010,pib_per_capita_2021,densidade_demografica_2010,cluster_0,cluster_1,cluster_2,cluster_3
1,ABADIA DOS DOURADOS - MG,MG,CIRO,PSD,0.575992,3188.775510,0.689,90.88,28184.630102,7.127400,False,False,True,False
2,ABADIANIA - GO,GO,DR ITAMAR,PP,0.619504,3481.894150,0.689,89.16,25842.850511,16.509351,False,False,True,False
3,ABAETE - MG,MG,IVANIR,MDB,0.491826,1764.057332,0.698,91.28,23869.415656,12.479830,False,False,True,False
4,ABAETETUBA - PA,PA,FRANCINETI CARVALHO,MDB,0.417201,2528.605204,0.628,86.29,12322.938529,98.228786,False,False,True,False
7,ABARE - BA,BA,EMANOEL DA FARMÁCIA,PT,0.823769,21543.171382,0.575,73.15,11823.686150,10.988425,True,False,False,False


In [ ]:
sns.set(style='whitegrid')
plt.figure(figsize=(6,4))
sns.scatterplot(data=df,
                x='emendas_pix_per_capita_partido_prefeito_eleito',
                y='porcentagem_votos_validos_2024',
                alpha=0.5)
plt.xlabel('Emendas PIX per capita')
plt.ylabel('Votos válidos (proporção)')
plt.title('Distribuição dos dados')
plt.show()

In [56]:
formula = 'porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito + C(sigla_partido_prefeito_eleito) + cluster_0 + cluster_1 + cluster_2 + cluster_3'
ols_res = smf.ols(formula, data=df).fit(cov_type='HC3')
print(ols_res.summary())

                                  OLS Regression Results                                  
Dep. Variable:     porcentagem_votos_validos_2024   R-squared:                       0.053
Model:                                        OLS   Adj. R-squared:                  0.044
Method:                             Least Squares   F-statistic:                     1339.
Date:                            Sun, 15 Jun 2025   Prob (F-statistic):               0.00
Time:                                    18:57:47   Log-Likelihood:                 1691.3
No. Observations:                            2307   AIC:                            -3339.
Df Residuals:                                2285   BIC:                            -3212.
Df Model:                                      21                                         
Covariance Type:                              HC3                                         
                                                        coef    std err          z      P>

e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 22, but rank is 21
  warnings.warn('covariance of constraints does not have full '


In [57]:
p_val = ols_res.pvalues.get('emendas_pix_per_capita_partido_prefeito_eleito', 1)
coef = ols_res.params.get('emendas_pix_per_capita_partido_prefeito_eleito', float('nan'))
if p_val < 0.05:
    direction = 'positiva' if coef > 0 else 'negativa'
    print(f'O efeito das Emendas PIX per capita é {direction} e estatisticamente significativo (coef={coef:.3f}, p={p_val:.3f})')
else:
    print(f'Nenhuma associação significativa encontrada (coef={coef:.3f}, p={p_val:.3f})')


O efeito das Emendas PIX per capita é positiva e estatisticamente significativo (coef=0.000, p=0.003)


In [19]:
# Modelo nulo
null_md = smf.mixedlm('porcentagem_votos_validos_2024 ~ 1', df, groups=df['sigla_partido_prefeito_eleito']).fit()

# Interceptos aleatórios
ri_md = smf.mixedlm('porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito',
                    df, groups=df['sigla_partido_prefeito_eleito']).fit()

# Intercepto e inclinação aleatórios
ris_md = smf.mixedlm('porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito',
                     df, groups=df['sigla_partido_prefeito_eleito'],
                     re_formula='~emendas_pix_per_capita_partido_prefeito_eleito').fit()

# Modelo completo com clusters
full_md = smf.mixedlm('porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito + cluster_0 + cluster_1 + cluster_2 + cluster_3',
                      df, groups=df['sigla_partido_prefeito_eleito'],
                      re_formula='~emendas_pix_per_capita_partido_prefeito_eleito').fit()

for lbl, m in [('Nulo', null_md), ('RI', ri_md), ('RI+RS', ris_md), ('Completo', full_md)]:
    print('', lbl)
    display(m.summary())

e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warning

 Nulo


e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2206: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2218: ConvergenceWarning: Gradient optimization failed, |grad| = 88.106246
  warnings.warn(msg, ConvergenceWarning)
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
e:\Learning\TCC-MBA\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix 

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Mixed Linear Model Regression Results
============================================================================
Model:            MixedLM Dependent Variable: porcentagem_votos_validos_2024
No. Observations: 4922    Method:             REML                          
No. Groups:       23      Scale:              0.0146                        
Min. group size:  2       Log-Likelihood:     3402.1582                     
Max. group size:  790     Converged:          Yes                           
Mean group size:  214.0                                                     
--------------------------------------------------------------------------------
                  Coef.     Std.Err.        z        P>|z|     [0.025     0.975]
--------------------------------------------------------------------------------
Intercept         0.593        0.003     172.271     0.000      0.587      0.600
Group Var         0.000        0.001                                            
============================================================================

"""

 RI


<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Mixed Linear Model Regression Results
=========================================================================================
Model:                 MixedLM     Dependent Variable:     porcentagem_votos_validos_2024
No. Observations:      4922        Method:                 REML                          
No. Groups:            23          Scale:                  0.0143                        
Min. group size:       2           Log-Likelihood:         3437.1167                     
Max. group size:       790         Converged:              Yes                           
Mean group size:       214.0                                                             
-----------------------------------------------------------------------------------------
                                               Coef. Std.Err.    z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      0.587    0.004 161.179 0.000  0.579  0.594
emendas_pix_per_capita_partido_prefeito_eleito 0.000    0.000  10.056 0.000  0.000  0.000
Group Var                                      0.000    0.001                            
=========================================================================================

"""

 RI+RS


<class 'statsmodels.iolib.summary2.Summary'>
"""
                               Mixed Linear Model Regression Results
====================================================================================================
Model:                    MixedLM         Dependent Variable:         porcentagem_votos_validos_2024
No. Observations:         4922            Method:                     REML                          
No. Groups:               23              Scale:                      0.0142                        
Min. group size:          2               Log-Likelihood:             3305.5309                     
Max. group size:          790             Converged:                  Yes                           
Mean group size:          214.0                                                                     
----------------------------------------------------------------------------------------------------
                                                           Coef. Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------------------------------
Intercept                                                  0.584    0.006 93.648 0.000  0.571  0.596
emendas_pix_per_capita_partido_prefeito_eleito             0.000    0.001  0.001 0.999 -0.002  0.002
Group Var                                                  0.001                                    
Group x emendas_pix_per_capita_partido_prefeito_eleito Cov 0.000                                    
emendas_pix_per_capita_partido_prefeito_eleito Var         0.000                                    
====================================================================================================

"""

 Completo


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                          Mixed Linear Model Regression Results
=========================================================================================================================
Model:                           MixedLM                Dependent Variable:                porcentagem_votos_validos_2024
No. Observations:                4922                   Method:                            REML                          
No. Groups:                      23                     Scale:                             0.0140                        
Min. group size:                 2                      Log-Likelihood:                    3309.9920                     
Max. group size:                 790                    Converged:                         No                            
Mean group size:                 214.0                                                                                   
-------------------------------------------------------------------------------------------------------------------------
                                                           Coef.    Std.Err.     z   P>|z|      [0.025         0.975]    
-------------------------------------------------------------------------------------------------------------------------
Intercept                                                  0.545 539213273.818 0.000 1.000 -1056838596.125 1056838597.215
cluster_0[T.True]                                          0.054 539213273.818 0.000 1.000 -1056838596.617 1056838596.724
cluster_1[T.True]                                          0.063 539213273.818 0.000 1.000 -1056838596.607 1056838596.733
cluster_2[T.True]                                          0.025 539213273.818 0.000 1.000 -1056838596.645 1056838596.696
cluster_3[T.True]                                          0.063 539213273.818 0.000 1.000 -1056838596.608 1056838596.733
emendas_pix_per_capita_partido_prefeito_eleito             0.000         0.005 0.000 1.000          -0.011          0.011
Group Var                                                  0.001                                                         
Group x emendas_pix_per_capita_partido_prefeito_eleito Cov 0.000                                                         
emendas_pix_per_capita_partido_prefeito_eleito Var         0.001                                                         
=========================================================================================================================

"""

In [20]:
p_val = full_md.pvalues.get('emendas_pix_per_capita_partido_prefeito_eleito', 1)
coef = full_md.params.get('emendas_pix_per_capita_partido_prefeito_eleito', float('nan'))
if p_val < 0.05:
    direction = 'positiva' if coef > 0 else 'negativa'
    print(f'No modelo multinível completo o efeito é {direction} e estatisticamente significativo (coef={coef:.3f}, p={p_val:.3f})')
else:
    print(f'No modelo multinível completo não há associação significativa (coef={coef:.3f}, p={p_val:.3f})')


No modelo multinível completo não há associação significativa (coef=0.000, p=1.000)
